In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder,normalize
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics



In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 

from sklearn.cluster import KMeans
from sklearn import model_selection
import xgboost as xgb
from xgboost import plot_importance



# 1. Data preprocessing

In [3]:
data_df = pd.read_csv('https://raw.githubusercontent.com/jyyang5/MAP569-project/master/CreditTraining.csv')
label_df = data_df['Prod_Category'].tolist()

# divide into categorical data and else as clean
clean_df = data_df.select_dtypes(exclude=['object']).copy()
cat_df = data_df.select_dtypes(include=['object']).copy()

# replace comma with dot 
temp_list = []
for ele in data_df['Net_Annual_Income'].tolist():
  if type(ele) is not float:
    temp_list.append(ele.replace(',','.'))
  else:
    temp_list.append(ele)

clean_df['Net_Annual_Income'] = temp_list
del cat_df['Net_Annual_Income']
clean_df[:2]

,Id_Customer,Y,Number_Of_Dependant,Years_At_Residence,Years_At_Business,Nb_Of_Products,Net_Annual_Income
0,7440,0,3.0,1,1.0,1,36
1,573,0,0.0,12,2.0,1,18


**Observations**: there are categorical and timestamps that is not numeric.

## 1.1. Deal with categorical data (timestamp data excluded)

From all categorical datas ['Customer_Type', 'BirthDate', 'Customer_Open_Date', 'P_Client',
       'Educational_Level', 'Marital_Status', 'Net_Annual_Income',
       'Prod_Sub_Category', 'Prod_Decision_Date', 'Source',
       'Type_Of_Residence', 'Prod_Closed_Date', 'Prod_Category']

We first exclude ['BirthDate', 'Customer_Open_Date', 'Net_Annual_Income', 'Prod_Decision_Date']

We use 0-1 encoding for each category

In [ ]:
for name in ['Customer_Type', 'P_Client',
            'Educational_Level', 'Marital_Status',
            'Prod_Sub_Category', 'Source',
            'Type_Of_Residence', 'Prod_Category']:
        print(cat_df[name].value_counts())

Since 'Customer_Type', 'P_Client', 'Source' all have just two types we therefore restrict ourselves to binary variable.

$$
Customer\_Type = 
 \begin{cases}
1& \text{Existing Client} \\
0 & \text{otherwise}
\end{cases} \\
P\_Client = 
 \begin{cases}
1& \text{P Client} \\
0 & \text{NP Client}
\end{cases} \\
Source= 
 \begin{cases}
1& \text{Sales} \\
0 & \text{Brranch}
\end{cases}
$$



In [ ]:
# add categorical data [timestamp data not added]
clean_df1 = clean_df.join(pd.get_dummies(cat_df[['Customer_Type', 'P_Client',
                                                'Educational_Level', 'Marital_Status',
                                                'Prod_Sub_Category', 'Source',
                                                'Type_Of_Residence', 'Prod_Category']]))

clean_df1['Customer_Type'] = clean_df1['Customer_Type_Existing Client']
del clean_df1['Customer_Type_Existing Client']
del clean_df1['Customer_Type_Non Existing Client']

clean_df1['P_Client'] = clean_df1['P_Client_NP_Client']
del clean_df1['P_Client_NP_Client']
del clean_df1['P_Client_P_Client']

clean_df1['Source'] = clean_df1['Source_Branch']
del clean_df1['Source_Branch']
del clean_df1['Source_Sales']


clean_df1[:2]

## 1.2. Deal with timestamps 
**Appraoches**: We use duration so that the variable is comparable 

['BirthDate', 'Customer_Open_Date', 'Net_Annual_Income', 'Prod_Decision_Date', 'Prod_Closed_Date']


- `Birth_Duration = Now - BirthDate` ('BirthDate'): assuming there is a distribution of credibility, just started working -> less credit, worked for a long time but not close to retirement -> high credit) 
- `Customer_Open_Duration = Now - Customer_Open_Date` ('Customer_Open_Date'): usually longer the history is, the more royal the customer is
- 'Prod_Closed_Date' - 'Prod_Decision_Date'
    - length of the product?: if product closed 
    - `Prod_not_closed = (Prod_closed != nan)`: dummy variable if the product is closed 'Prod_not_closed' = 0 if closed
    - `Prod_Decision_Duration = Now - Prod_Decision_Date`


In [ ]:
# duration: birth - now
temp_list = [(datetime.now().date() - datetime.strptime(datetime_str, '%d/%m/%Y').date()).days for datetime_str in cat_df['BirthDate'].tolist()]
clean_df1['Birth_Duration'] = temp_list

# duration: Customer_Open_Date
temp_list = [(datetime.now().date() - datetime.strptime(datetime_str, '%d/%m/%Y').date()).days for datetime_str in cat_df['Customer_Open_Date'].tolist()]
clean_df1['Customer_Open_Duration'] = temp_list

# dummy var: product closed = 1
temp_list = [int(type(ele) != float) for ele in cat_df['Prod_Closed_Date'].tolist()]
clean_df1['Prod_not_closed'] = temp_list

# duration: Prod_Decision_Date
temp_list = [(datetime.now().date() - datetime.strptime(datetime_str, '%d/%m/%Y').date()).days for datetime_str in cat_df['Prod_Decision_Date'].tolist()]
clean_df1['Prod_Decision_Duration'] = temp_list

# convert to a list 
data_list = []
labels = []
for name in clean_df1.columns.values:
    if name is 'Id_Customer':
        pass
    elif name is 'Y':
        labels = clean_df1[name].tolist()
    else:
        data_list.append(clean_df1[name].tolist())

data_list = np.transpose(data_list)
data_list = np.array(data_list, dtype=np.float64)
np.where(np.isnan(data_list))

Before we proceed to training and testing, we check if there is nan element

Customers with nan 
- Years_At_Business
    - Id_Customer = 398, 5882	
- Number_Of_Dependant
    - Id_Customer = 8953, 9588	
- Net_Annual_Income
    - Id_Customer = 9399, 9555	

## 1.3. Nan elemnts 
There are two usual approaches to deal with nan elements, the first one drops the data with nan, the second on the other hand replace the nan element. Since it is very likely that we may face future clients with unkown data (nan), so that we use the second approach to make the method proposed more general.

- Median 
- Most frequent 

In [ ]:
clean_df2 = clean_df1.copy()

for name in ['Number_Of_Dependant', 'Years_At_Business', 'Net_Annual_Income']:
    temp_list = np.array(clean_df2[name].tolist(), dtype=np.float64)
    temp_list[np.isnan(temp_list)] = np.median(temp_list[~np.isnan(temp_list)])
    # print(sum(np.isnan(temp_list)))
    clean_df2[name] = temp_list

# convert to train mat 
data_list = []
labels = []
for name in clean_df2.columns.values:
    if name == 'Id_Customer':
        pass
        
    elif name is 'Y':
        labels = clean_df2[name].tolist()
    else:
        data_list.append(clean_df2[name].tolist())

data_list = np.transpose(data_list)
data_list = np.array(data_list, dtype=np.float64)
np.where(np.isnan(data_list))
train_df = clean_df2.copy()
del train_df['Y']
del train_df['Id_Customer']
'Check done: ', np.shape(train_df)== np.shape(data_list)

# 2. Models 

In [ ]:
# split data
X_train, X_test, y_train, y_test = train_test_split(data_list, labels, test_size=0.25, random_state=42)

# np.save('X_train.npy',X_train)
# np.save('X_test.npy',X_test)
# np.save('y_train.npy',y_train)
# np.save('y_test.npy',y_test)

print('-----------all-----------')
for i in set(labels):
    print(i,labels.count(i))
print('percentage of 1:', labels.count(1)/len(labels))

print('-----------train-----------')
for i in set(labels):
    print(i,y_train.count(i))
print('percentage of 1:', y_train.count(1)/len(y_train))

print('-----------test-----------')
for i in set(labels):
    print(i,y_test.count(i))
print('percentage of 1:', y_test.count(1)/len(y_test))


The data seems to be unbalanced (two labels has a relative big difference in percentag), we first try without balancing the data.

## 2.1. Evaluation metrics

| 		|     same clusters|   different clusters|
| :-------- | --------:| :------: |
| same class    |   TP  |  FN |
|different class|   FP  |  TN |
- $P(precision) = \frac{TP}{TP+FP}$ 
- $R(recall) = \frac{TP}{TP+FN}$ 
- $F_{\beta}= \frac{(\beta^2+1)PR}{\beta^2P+R}$

### Note: since label=1 means that the client has defaulted on its credit, which is something that we definately want to avoid, we focus on *R(recall)* (percentage of detected clients among all truly defaulted clients) 


## 2.2. Models
Here we use two types of models 
- **Intrinsic**: if after feature engineering, the data itself could seperate.
    - KNN
    - Kmeans
    - SVM
- **Instrumental**: might contain some sematic meaning (relationshop), that is not explict so that we need more complex models. 
    - Random forest 
    - XGBoost 

In [ ]:
def convert_lables(labels):
    """ convert the input (0/1) labels to what is needed 
    return a binary label sequence 
    """
    labels = np.array(labels)
    if sum(labels) > len(labels)/2:
        labels[labels==1] = -1
        labels[labels==0] = 1
        labels[labels==-1] = 0
    return labels

def eval_metric(y_test, preds):
    """ 
    print the classification report 
    """
    
    preds = convert_lables(preds)
    print('confusion matrix')
    print(confusion_matrix(y_test, preds))
    print('\n')
    print('summary')
    print(classification_report(y_test, preds))
    fpr, tpr, thresholds = metrics.roc_curve(y_test, preds, pos_label=2)
    metrics.auc(fpr, tpr)

def test_deterministic(X_train, y_train):
    """ print the evaluation results of deterministic models
    """
    # KNeighbors
    neigh = KNeighborsClassifier(n_neighbors=2)
    neigh.fit(X_train, y_train)
    pred_KNN = neigh.predict(X_test)
    print('----------------KNeighbors-------------------')
    eval_metric(y_test, pred_KNN)

    # Kmeans
    neigh = KMeans(n_clusters=2)
    neigh.fit(X_train, y_train)
    pred_Kmeans = convert_lables(neigh.predict(X_test))
    print('----------------Kmeans-------------------')
    eval_metric(y_test, pred_Kmeans)

    # SVM
    clf = SVC(gamma='auto')
    clf.fit(X_train, y_train)
    pred_SVM = convert_lables(clf.predict(X_test))
    print('----------------SVM-------------------')
    eval_metric(y_test, pred_SVM)

In [ ]:
def test_non_deterministic(X_train, y_train):
    """ print the evaluation results of non-deterministic models
    """
    ################################
    # random forest model 
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)
    # predictions
    pred_rfc = rfc.predict(X_test)
    print('----------------randomForest-------------------')
    eval_metric(pred_rfc, y_test)

    rfc_importances = rfc.feature_importances_
    rfc_std = np.std([rfc.feature_importances_ for tree in rfc.estimators_], axis=0)
    rfc_indices = np.argsort(rfc_importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X_train.shape[1]):
        print("%d. feature %s (%f)" % (f + 1, train_df.columns.tolist()[rfc_indices[f]], rfc_importances[rfc_indices[f]]))
    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X_train.shape[1]), rfc_importances[rfc_indices],
        color="r", yerr=rfc_std[rfc_indices], align="center")
    plt.xticks(range(X_train.shape[1]), rfc_indices)
    plt.xlim([-1, X_train.shape[1]])
    plt.show()

    ################################
    # XGBoost
    xg_reg = xgb.XGBRegressor(objective ='reg:logistic') 
    xg_reg.fit(X_train, y_train)
    xg_preds = (xg_reg.predict(X_test)>=0.5)*1
    print('----------------XGBoost-------------------')
    eval_metric(xg_preds, y_test)
    # plot feature importance
    xg_importances = xg_reg.feature_importances_
    xg_indices = np.argsort(xg_importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X_train.shape[1]):
        print("%d. feature %s (%f)" % (f + 1, train_df.columns.tolist()[xg_indices[f]], xg_importances[xg_indices[f]]))

    plot_importance(xg_reg)
    plt.show()


In [ ]:
test_deterministic(X_train, y_train)

In [ ]:
test_non_deterministic(X_train, y_train)

**Observations**
- indeed, the recall reaches 0.76 and 0.63 for RF and XGboost even if the dataset is unbalanced
- from the feature importance, we can infer that the feature reconstructed is indeed helpful ('Prod_not_closed' is the most dominating feature in both models)

## 2.3. Balance data
In Section 2.2, we did some experiments but find that even if the overall accurancy is high, the prediction results of clients with label=1 is not good. Especially with classifiers like SVM, the class with more samples (majority class) is favoured. We therefore resample the minority class to balance the train data and see how the result might improve. Since the size of the minority class is very small, we w.l.o.g. use Upsampling.   

# 3. Encapsulation 
We combine all mentioned, add a few more functionalities and create the class below 

In [ ]:
class Credit_predictor():
    def __init__(self,path):
        """
        read the data
        """
        self.raw_data = pd.read_csv('CreditTraining.csv')
        self.clean_df = self.raw_data.select_dtypes(exclude=['object']).copy()
        self.cat_df = self.raw_data.select_dtypes(include=['object']).copy()
        # categorical data 
        self.cates = ['Customer_Type', 'P_Client',
            'Educational_Level', 'Marital_Status',
            'Prod_Sub_Category', 'Source',
            'Type_Of_Residence', 'Prod_Category']
        # all timestamp data 
        self.date_trans_set = {'Birth_Duration':'BirthDate',
             'Customer_Open_Duration':'Customer_Open_Date',
             'Prod_Decision_Duration':'Prod_Decision_Date'}
        
    def data_preprocessing_0(self):
        """
        seperate the data into numerical set and categorical set
        """
        self.clean_df = self.raw_data.select_dtypes(exclude=['object']).copy()
        self.cat_df = self.raw_data.select_dtypes(include=['object']).copy()
        temp_list = []
        # convert value from comma to dot 
        for ele in self.raw_data['Net_Annual_Income'].tolist():
            if type(ele) is not float:
                temp_list.append(ele.replace(',','.'))
            else:
                temp_list.append(ele)
        self.clean_df['Net_Annual_Income'] = temp_list
        del self.cat_df['Net_Annual_Income']
        self.dates_transformer()
        
    def dates_transformer(self):
        """
        processing for the date
        """
        # convert timestamp to duration: now - timestamp 
        for i in self.date_trans_set:
            temp_list = [(datetime.now().date() - datetime.strptime(datetime_str, '%d/%m/%Y').date()).days for datetime_str in self.cat_df[self.date_trans_set[i]].tolist()]
            self.clean_df[i] = temp_list
            del self.cat_df[self.date_trans_set[i]]
        # indicator whether product is closed 
        temp_list = [int(type(ele) != float) for ele in self.cat_df['Prod_Closed_Date'].tolist()]
        self.clean_df['Prod_not_closed'] = temp_list
        del self.cat_df['Prod_Closed_Date']

    def data_preprocessing_simple(self):
        self.data_simple = self.clean_df.copy()
        for i in self.cates:
            labelencoder = LabelEncoder()
            labelencoder.fit(self.cat_df[i])
            self.data_simple[i] = labelencoder.transform(self.cat_df[i])
        
    def data_preprocessing_onehot(self):
        binary = ['Customer_Type', 'P_Client', 'Source']
        no_binary = [i for i in self.cates if i not in binary]
        self.data_oh = self.clean_df.copy()
        for i in binary:
            labelencoder = LabelEncoder()
            labelencoder.fit(self.cat_df[i])
            """
            To see the representation of the labels : list(labelencoder.classes_)
            """
            self.data_oh[i] = labelencoder.transform(self.cat_df[i])
        self.data_oh = self.data_oh.join(pd.get_dummies(self.cat_df[no_binary]))
        
    def split_dataset(self,dataset):
        X = dataset.drop('Y',axis = 1)
        Y = dataset.Y
        try:
            imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
            imp_mean.fit(X)
        except:
            imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
            imp_mean.fit(X)
        '''
        SimpleImputer:
        strategy : mean, median, most_frequent, constant
        '''
        X_imputed = imp_mean.transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X_imputed, Y, test_size=0.25, random_state=42)
        return X_train, X_test, y_train, y_test
    
    
    def split_dataset_4_imbal(self,dataset):
        X_train, X_test, y_train, y_test = self.split_dataset(dataset)
        ros = RandomOverSampler(random_state=0)
        X_train, y_train = ros.fit_resample(X_train, y_train)
   #     ros = RandomOverSampler(random_state=0)
   #     X_test, y_test = ros.fit_resample(X_test, y_test)
        return X_train, X_test, y_train, y_test
        
    
    def convert_labels(self,labels):
        """ 
        convert the input (0/1) labels to what is needed 
        return a binary label sequence 
        """
        labels = np.array(labels)
        if sum(labels) > len(labels)/2:
            labels[labels==1] = -1
            labels[labels==0] = 1
            labels[labels==-1] = 0
        return labels

   
    def eval_metric(self, y_test, preds):
        """ 
        print the classification report 
        """
     #   preds = convert_labels(preds)
        print('confusion matrix')
        print(confusion_matrix(y_test, preds))
        print('\n')
        print('summary')
        print(classification_report(y_test, preds))
        fpr, tpr, thresholds = metrics.roc_curve(y_test, preds, pos_label=2)
        metrics.auc(fpr, tpr)


In [ ]:
path = 'CreditTraining.csv'
Cp = Credit_predictor(path)
Cp.data_preprocessing_0()
Cp.data_preprocessing_simple()
Cp.data_preprocessing_onehot()

In [ ]:
# one-hot
X_train, X_test, y_train, y_test = Cp.split_dataset(Cp.data_oh)
# one-hot [balanced]
X_train_b, X_test_b, y_train_b, y_test_b = Cp.split_dataset_4_imbal(Cp.data_oh)
# categorical -> 0:n_classes-1
X_train_t, X_test_t, y_train_t, y_test_t = Cp.split_dataset(Cp.data_simple)
# categorical -> 0:n_classes-1 [balanced]
X_train_tb, X_test_tb, y_train_tb, y_test_tb = Cp.split_dataset_4_imbal(Cp.data_simple)

In [ ]:
def model_eval(model, Cp):
    """
    print the evaluation result of the model with the Cp object 
    """

    print('----------------one-hot-------------------')
    model.fit(X_train, y_train)
    pred_y = model.predict(X_test)
    Cp.eval_metric(y_test, pred_y)
    print('----------------one-hot[balanced]-------------------')
    model.fit(X_train_b, y_train_b)
    pred_y_b = model.predict(X_test_b)
    Cp.eval_metric(y_test_b, pred_y_b)
    print('----------------0:n_classes-1-------------------')
    model.fit(X_train_t, y_train_t)
    pred_y_t = model.predict(X_test_t)
    Cp.eval_metric(y_test_t, pred_y_t)
    print('----------------0:n_classes-1[balanced]-------------------')
    model.fit(X_train_tb, y_train_tb)
    pred_y_tb = model.predict(X_test_tb)
    Cp.eval_metric(y_test_tb, pred_y_tb)


## 3.1. Model selection 
### 3.1.1 LogisticRegression

In [ ]:
lr = LogisticRegression(solver='liblinear')
model_eval(lr, Cp)

### 3.1.2 Random Forest 

In [ ]:
rfc = RandomForestClassifier(random_state=100)
model_eval(rfc, Cp)

### 3.1.3. SVM

In [ ]:
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)
pred_SVM = clf.predict(X_test)

Cp.eval_metric(y_test, pred_SVM)

In [ ]:
X_train_n = normalize(X_train_b, norm='l2')
X_test_n = normalize(X_test_b, norm='l2')

clf = SVC(gamma='auto')
clf.fit(X_train_n, y_train_b)
pred_SVM_b = clf.predict(X_test_n)

Cp.eval_metric(y_test_b, pred_SVM_b)

### 3.1.4. LDA

In [ ]:
LDA = LinearDiscriminantAnalysis()
model_eval(LDA, Cp)LDA = LinearDiscriminantAnalysis()
model_eval(LDA, Cp)

### 3.1.5. XGBoost 

In [ ]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic')
model_eval(xg_reg, Cp)

In [ ]:
xg_tweedie = xgb.XGBClassifier(objective ='binary:logitraw', eval_metric = 'error@0.5')
model_eval(xg_tweedie, Cp)


Changing objective and evaluation metric do not seem to have a huge difference.

**Observations**
- Balancing the data indeed helps to improve the model performance 
- We **select logistiRegression, LDA, and XGBoost given their $R_{\text{label=1}}$ around 0.85, $P_{\text{label=0}}=0.99$, $P_{\text{label=1}}$ around 0.42 using the two balanced data.**


## 3.2. Parameter Tuning 

In this section we perform more fine-grined parametr tuning for the three models mentioned.

$\color{red}{\text{TODO:}}$


# 4. Voting  

In this section, we propose two voting methods that utilize the predictions of several models and hopes it would maximize the information obatined so far.
- Improve precision of predicted defaulted
- Improve coverage of truly defaulted

## 4.1. Minimize innocent (Improve precision of predicted defaulted)

Since we have three models **logistiRegression, LDA, and XGBoost** using balanced data have $R_{\text{label=1}}$ around 0.85, meaning most of the defaulted customers are detected (which we think is a **relative good coverage**).

But $\text{label=1} \approx 0.42$ , meaning more than half of the predicted is not defaulted, and it would be costly to do a thorough investigation of the users so we want to **minimize those innocent but detected**.   

With 3 predictions from 3 individual parties, we want to extract the mutual information and believing that if one client is predictd defaulted by all parties then there is a larger probability that the one is defaulted compared with (1/3 of the parties voted so). And of course, we might lose some covergae. **We try taking a logical AND to do this**.

**Essentially, we are doing a voting by setting the weight to be 100% iff. all parties vote 1, ensuring that the number of inocent client is minimized**


In [ ]:
lr.fit(X_train_tb, y_train_tb)
lr_pred_y_tb = lr.predict(X_test_tb)
Cp.eval_metric(y_test_tb, lr_pred_y_tb)

xg_reg.fit(X_train_tb, y_train_tb)
xg_pred_y_tb = xg_reg.predict(X_test_tb)
Cp.eval_metric(y_test_tb, xg_pred_y_tb)

LDA.fit(X_train_tb, y_train_tb)
LDA_pred_y_tb = LDA.predict(X_test_tb)
Cp.eval_metric(y_test_tb, LDA_pred_y_tb)

In [ ]:
Cp.eval_metric(y_test_tb, LDA_pred_y_tb & xg_pred_y_tb & lr_pred_y_tb)

**Observation**: we did increase $P_{\text{label=1}}$ to 0.44, but $R_{\text{label=1}}$ drops to 0.84, the improvement of $P_{\text{label=1}}$ is small, one potential explaination is that the three parties share a lot of mutual information. 

In [ ]:
from scipy.spatial import distance

parties_pred_mat = [LDA_pred_y_tb, xg_pred_y_tb, lr_pred_y_tb]
distance.cdist(parties_pred_mat, parties_pred_mat, 'hamming')

Indeed, the *Minimize innocent* voting mechanism fail to improve a $P_{\text{label=1}}$ a lot is largely due to the percentage of shared information is dominant for all three parties (around 1% difference).

## 4.2. Add trustworthy information ( Improve coverage of truly defaulted)

We use one of the three parties as a base (with good covergae) and then we add prediction by party with high $P_{\text{label=1}}$ but possibly low coverage ($R_{\text{label=1}}$), so as to improve precison and recall at the same time.

**Essentially, we are doing a voting by setting the weight of trustworthy party to 100%, meaning once they vote 1, the client is 1 in prediction.**

- Base
    - `lr_pred_y_tb`: $P_{\text{label=1}}=0.43, R_{\text{label=1}}=0.88$ 
    - `xg_pred_y_tb`: $P_{\text{label=1}}=0.43, R_{\text{label=1}}=0.85$ 
- Trustworthy
    - `lr_pred_y` : $P_{\text{label=1}}=0.80, R_{\text{label=1}}=0.15$ 
    - `rfc_pred_y` : $P_{\text{label=1}}=0.76, R_{\text{label=1}}=0.28$       

In [ ]:
rfc.fit(X_train, y_train)
rfc_pred_y = rfc.predict(X_test)

lr.fit(X_train, y_train)
lr_pred_y = lr.predict(X_test)


print('-------------------LDA+lr-------------------')
Cp.eval_metric(y_test_tb, lr_pred_y | LDA_pred_y_tb)

print('-------------------LDA+rfc-------------------')
Cp.eval_metric(y_test_tb, rfc_pred_y | LDA_pred_y_tb)

print('-------------------LDA+rfc+lr-------------------')
Cp.eval_metric(y_test_tb, lr_pred_y | rfc_pred_y | LDA_pred_y_tb)

In [ ]:
print('-------------------XG+lr-------------------')
Cp.eval_metric(y_test_tb, lr_pred_y | xg_pred_y_tb)

print('-------------------XG+rfc-------------------')
Cp.eval_metric(y_test_tb, rfc_pred_y | xg_pred_y_tb)

print('-------------------XG+rfc+lr-------------------')
Cp.eval_metric(y_test_tb, lr_pred_y | rfc_pred_y | xg_pred_y_tb)

Unfortunately, these *trustworthy information* seem to have been covered already by the parties with good coverage.

# 5. Conclusion 
